In [17]:
import pandas as pd
import numpy as np
from features_creation import window_sliding, save_data

In [18]:
data = pd.read_csv("data/train.csv")

targets = data.columns[-3:]

In [19]:
data_cp = data.copy()

In [20]:
sensors = data_cp.columns[4:9]

Зададим размеры окон в часах, основываясь на выделенных периодах. Воспользуемся функциями min, max, std. При необходимости можно изменить периоды или функции.

In [21]:
window_sliding(data_cp, dict(zip(list(targets) + list(sensors), [{np.max: [24*20, 24*6, 12], np.min: [24*20, 24*6, 12], np.std: [24*20, 24*6, 12], np.average: [24*20, 24*6, 12]}, 
                                                           {np.max: [24*20, 24*6, 12], np.min: [24*20, 24*6, 12], np.std: [24*20, 24*6, 12], np.average: [24*20, 24*6, 12]}, 
                                                           {np.max: [24*20, 24*6, 12], np.min: [24*20, 24*6, 12], np.std: [24*20, 24*6, 12], np.average: [24*20, 24*6, 12]},
                                                                {np.max: [24*20, 24*6, 12], np.min: [24*20, 24*6, 12], np.std: [24*20, 24*6, 12], np.average: [24*20, 24*6, 12]},
                                                                {np.max: [24*20, 24*6, 12], np.min: [24*20, 24*6, 12], np.std: [24*20, 24*6, 12], np.average: [24*20, 24*6, 12]},
                                                                {np.max: [24*20, 24*6, 12], np.min: [24*20, 24*6, 12], np.std: [24*20, 24*6, 12], np.average: [24*20, 24*6, 12]},
                                                                {np.max: [24*20, 24*6, 12], np.min: [24*20, 24*6, 12], np.std: [24*20, 24*6, 12], np.average: [24*20, 24*6, 12]},
                                                                {np.max: [24*20, 24*6, 12], np.min: [24*20, 24*6, 12], np.std: [24*20, 24*6, 12], np.average: [24*20, 24*6, 12]}])), in_place=True)

In [22]:
data_cp.head(5)

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,...,sensor_5_amax_12h,sensor_5_amin_480h,sensor_5_amin_144h,sensor_5_amin_12h,sensor_5_std_480h,sensor_5_std_144h,sensor_5_std_12h,sensor_5_average_480h,sensor_5_average_144h,sensor_5_average_12h
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,...,1293.4,1293.4,1293.4,1293.4,NaN,NaN,NaN,1293.400000,1293.400000,1293.400000
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,...,1293.4,1010.9,1010.9,1010.9,199.757666,199.757666,199.757666,1152.150000,1152.150000,1152.150000
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,...,1293.4,1010.9,1010.9,1010.9,142.700397,142.700397,142.700397,1140.433333,1140.433333,1140.433333
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,...,1293.4,1010.9,1010.9,1010.9,131.694783,131.694783,131.694783,1171.125000,1171.125000,1171.125000
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,...,1293.4,1010.9,1010.9,1010.9,115.371868,115.371868,115.371868,1163.340000,1163.340000,1163.340000


Также можно продифферецнировать ряд

In [23]:
def diff(targets_with_n: dict):
    for t, n in targets_with_n.items():
        data_cp[f'{t}_diff_{n}'] = data_cp[f'{t}'].diff(n)

In [24]:
diff(dict(zip(targets, [1, 1, 1])))

In [25]:
diff(dict(zip(targets, [2, 2, 2])))

Также сделаем отдельные параметры для дня, месяца, года. Также сделаем отдельный параметр для сезонов, как это было сделано в EDA

In [26]:
data_cp.date_time = pd.to_datetime(data.date_time)

In [27]:
interval_1 = data.date_time < data.date_time[3450]
interval_2 = (data.date_time > data.date_time[3450]) & (data.date_time < data.date_time[4200])
interval_3 = data.date_time > data.date_time[4200]

data_cp["season"] = np.nan
data_cp.loc[interval_1, 'season'], data_cp.loc[interval_2, 'season'], data_cp.loc[interval_3, 'season'] = 1, 2, 3

In [28]:
data_cp["date_time_hour"] = data_cp.date_time.dt.hour.astype('float64')
data_cp["date_time_day"] = data_cp.date_time.dt.day.astype('float64')
data_cp["date_time_month"] = data_cp.date_time.dt.month.astype('float64')
data_cp["date_time_year"] = data_cp.date_time.dt.year.astype('float64')

In [29]:
data_cp.iloc[:, 9:].head(5)

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,target_carbon_monoxide_amax_480h,target_carbon_monoxide_amax_144h,target_carbon_monoxide_amax_12h,target_carbon_monoxide_amin_480h,target_carbon_monoxide_amin_144h,target_carbon_monoxide_amin_12h,target_carbon_monoxide_std_480h,...,target_benzene_diff_1,target_nitrogen_oxides_diff_1,target_carbon_monoxide_diff_2,target_benzene_diff_2,target_nitrogen_oxides_diff_2,season,date_time_hour,date_time_day,date_time_month,date_time_year
0,2.5,12.0,167.7,2.5,2.5,2.5,2.5,2.5,2.5,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,18.0,10.0,3.0,2010.0
1,2.1,9.9,98.9,2.5,2.5,2.5,2.1,2.1,2.1,0.282843,...,-2.1,-68.8,NaN,NaN,NaN,1.0,19.0,10.0,3.0,2010.0
2,2.2,9.2,127.1,2.5,2.5,2.5,2.1,2.1,2.1,0.208167,...,-0.7,28.2,-0.3,-2.8,-40.6,1.0,20.0,10.0,3.0,2010.0
3,2.2,9.7,177.2,2.5,2.5,2.5,2.1,2.1,2.1,0.173205,...,0.5,50.1,0.1,-0.2,78.3,1.0,21.0,10.0,3.0,2010.0
4,1.5,6.4,121.8,2.5,2.5,2.5,1.5,1.5,1.5,0.367423,...,-3.3,-55.4,-0.7,-2.8,-5.3,1.0,22.0,10.0,3.0,2010.0


Также нам может понадобиться масштабирование признаков. Если этого не сделать, то в случае применения регуляризации модель будет работать хуже, так как у малых по модулю признаков, возможно, будут большие веса, а регуляризация "штрафует" за это. Также масштабирование повысит интерпретируемость, так как веса можно будет понимать как важность признака.

In [30]:
features = list(data_cp.columns[1:9]) + list(data_cp.columns[12:])

In [31]:
from sklearn.preprocessing import StandardScaler

X, Y = data_cp.dropna()[features + list(targets)], data_cp.dropna()[targets]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(X, Y)

In [32]:
save_data('data_v4.1', features_scaled, Y, data_cp)